In [26]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline

# Load Data

In [27]:
X_t0_path = "X_t0.parquet"
y_t0_path = "y_t0.parquet"

X_t1_path = "X_t1.parquet"
y_t1_path = "y_t1.parquet"

X_t2_path = "X_t2.parquet"
y_t2_path = "y_t2.parquet"

x_t3_path = "X_t3.parquet"

X_t0 = pd.read_parquet(X_t0_path)
y_t0 = pd.read_parquet(y_t0_path)

X_t1 = pd.read_parquet(X_t1_path)
y_t1 = pd.read_parquet(y_t1_path)

X_t2 = pd.read_parquet(X_t2_path)
y_t2 = pd.read_parquet(y_t2_path)

X_t3 = pd.read_parquet(x_t3_path)

# Join datasets

In [28]:
X_t0 = X_t0.reset_index(drop=True)
X_t1 = X_t1.reset_index(drop=True)
X_t2 = X_t2.reset_index(drop=True)
X_t3 = X_t3.reset_index(drop=True)

y_t0 = y_t0.reset_index(drop=True)
y_t1 = y_t1.reset_index(drop=True)
y_t2 = y_t2.reset_index(drop=True)

In [29]:
X = pd.concat([X_t0, X_t1, X_t2], axis=0)
X = X.reset_index(drop=True)

y = pd.concat([y_t0, y_t1, y_t2], axis=0)
y = y.reset_index(drop=True)

assert X.columns.equals(X_t0.columns)
assert X.columns.equals(X_t1.columns)
assert X.columns.equals(X_t2.columns)
assert X.columns.equals(X_t3.columns)

In [30]:
y.value_counts()

target
0         213160
1         141208
Name: count, dtype: int64

In [31]:
X.dtypes.value_counts()


float64    60
int64      16
object      1
Name: count, dtype: int64

In [32]:
X

,borrow_block_number,borrow_timestamp,wallet_address,first_tx_timestamp,last_tx_timestamp,wallet_age,incoming_tx_count,outgoing_tx_count,net_incoming_tx_count,total_gas_paid_eth,...,market_macdsignal,market_max_drawdown_365d,market_natr,market_plus_di,market_plus_dm,market_ppo,market_rocp,market_rocr,unique_borrow_protocol_count,unique_lending_protocol_count
0,7711117,1.557197e+09,0x502cb8985b2c92a8d4bf309cdaa89de9be442708,1.537224e+09,1.557197e+09,19973049.0,199,438,-239,0.397391,...,1.212219,-0.895911,4.479356,33.216622,33.415526,-2.370346,0.104294,1.104294,0,1
1,7711123,1.557197e+09,0x502cb8985b2c92a8d4bf309cdaa89de9be442708,1.537224e+09,1.557197e+09,19973188.0,200,439,-239,0.399063,...,1.212219,-0.895911,4.479356,33.216622,33.415526,-2.370346,0.104294,1.104294,1,1
2,7711126,1.557197e+09,0x502cb8985b2c92a8d4bf309cdaa89de9be442708,1.537224e+09,1.557197e+09,19973238.0,201,440,-239,0.400895,...,1.212219,-0.895911,4.479356,33.216622,33.415526,-2.370346,0.104294,1.104294,1,1
3,7711672,1.557205e+09,0xa7ff0d561cd15ed525e31bbe0af3fe34ac2059f6,1.557191e+09,1.557198e+09,13922.0,8,56,-48,0.604001,...,1.212219,-0.895911,4.479356,33.216622,33.415526,-2.370346,0.104294,1.104294,0,1
4,7712572,1.557217e+09,0xbd9ed130a53cfafcf81502e4d35329a6c4d53410,1.557217e+09,1.557217e+09,264.0,2,3,-1,0.003008,...,1.212219,-0.895911,4.479356,33.216622,33.415526,-2.370346,0.104294,1.104294,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354363,15269981,1.659537e+09,0x4196c40de33062ce03070f058922baa99b28157b,1.606233e+09,1.659268e+09,53303808.0,460,540,-80,10.015946,...,91.682257,-0.816810,7.693345,33.422749,501.062341,12.414672,0.063341,1.063341,2,2
354364,15269988,1.659537e+09,0xd155ae951285d54a8068d2b5ade9e00eb17e8947,1.637428e+09,1.659536e+09,22108825.0,111,159,-48,2.699685,...,91.682257,-0.816810,7.693345,33.422749,501.062341,12.414672,0.063341,1.063341,1,1
354365,15270006,1.659537e+09,0x7944e84d18803f926743fa56fb7a9bb9ba5f5f24,1.645629e+09,1.659535e+09,13908170.0,120,6055,-5935,215.453509,...,91.682257,-0.816810,7.693345,33.422749,501.062341,12.414672,0.063341,1.063341,1,1
354366,15270173,1.659540e+09,0x1212040f31e25d707bcd01971e76689678073214,1.651314e+09,1.659539e+09,8225703.0,65,72,-7,0.457836,...,91.682257,-0.816810,7.693345,33.422749,501.062341,12.414672,0.063341,1.063341,1,2


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline

In [34]:
from sklearn.base import BaseEstimator, TransformerMixin

temporal_columns = [
    "risky_first_tx_timestamp", 
    "time_since_last_liquidated"
]

log_columns = [
    "wallet_age",
    "incoming_tx_count",
    "outgoing_tx_count",
    "total_gas_paid_eth",
    "risky_tx_count",
    "risky_unique_contract_count",
    "outgoing_tx_sum_eth",
    "incoming_tx_sum_eth",
    "outgoing_tx_avg_eth",
    "risky_last_tx_timestamp",
    "incoming_tx_avg_eth",
    "max_eth_ever",
    "risk_factor",
    "total_collateral_eth",
    "total_collateral_avg_eth",
    "total_available_borrows_eth",
    "total_available_borrows_avg_eth",
    "risk_factor_above_threshold_daily_count",
    "avg_risk_factor",
    "borrow_amount_avg_eth",
    "borrow_count",
    "repay_amount_sum_eth",
    "repay_amount_avg_eth",
    "repay_count",
    "borrow_repay_diff_eth",
    "deposit_count",
    "deposit_amount_sum_eth",
    "liquidation_amount_sum_eth",
    "time_since_first_deposit",
    "withdraw_amount_sum_eth",
    "risky_first_last_tx_timestamp_diff",
    "avg_gas_paid_per_tx_eth",
    "risky_sum_outgoing_amount_eth",
    "min_eth_ever",
    "total_balance_eth",
    "avg_weighted_risk_factor",
    "max_risk_factor",
    "withdraw_deposit_diff_if_positive_eth",
    "borrow_amount_sum_eth",
]

outlier_columns = [
    "liquidation_count",
    "market_adx",
    "market_adxr",
    "market_correl",
    "market_linearreg_slope",
    "market_macd_macdext",
    "market_macd_macdfix",
    "market_macdsignal_macdfix",
    "market_macdsignal",
    "market_natr",
    "market_plus_dm",
    "market_apo",
    "market_ppo",
    "first_tx_timestamp",
    "net_incoming_tx_count",
    "market_macdsignal_macdext"
]


no_transform_columns = [
    "market_aroonosc",
    "market_atr",
    "market_cci",
    "market_cmo",
    "market_dx",
    "market_fastk",
    "market_macd",
    "market_macdsignal",
    "market_max_drawdown_365d",
    "market_plus_di",
    "market_rocp",
    "market_rocr",
    "last_tx_timestamp",
    "market_aroonup",
    "market_aroonosc",
    "market_fastd"
    ]

drop_columns = [
    "borrow_block_number",
    "borrow_timestamp",
    "wallet_address"]

categorical_columns = [
    "market_ht_trendmode",
    "unique_borrow_protocol_count",
    "unique_lending_protocol_count",
    ]


all_columns = temporal_columns + log_columns + outlier_columns + no_transform_columns + drop_columns + categorical_columns

for X_ti in [X_t0, X_t1, X_t2, X_t3]:

    for col in X_ti.columns:
        assert col in all_columns
    for col in all_columns:
        assert col in X_ti.columns

In [35]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

class TemporalTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_transform):
        self.columns_to_transform = columns_to_transform

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns_to_transform:
            X[f"has_{col}"] = (X[col] == 999999999).astype(int)
            X[col] = X[col].replace(999999999, 0)
        return X

In [36]:
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = np.log1p(X[col].clip(lower=0))
        return X

In [37]:
class OutlierTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns, lower_percentile=0.05, upper_percentile=0.95):
        self.columns = columns
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            lower_bound = X[col].quantile(self.lower_percentile)
            upper_bound = X[col].quantile(self.upper_percentile)
            X[col] = X[col].clip(lower_bound, upper_bound)
        return X

In [38]:
class DropTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop, errors='ignore')

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

temporal_transformer = Pipeline([
    ("temporal", TemporalTransformer(columns_to_transform=temporal_columns))
])

log_transformer = Pipeline([
    ("log", LogTransformer(columns=log_columns+temporal_columns))
])

outlier_transformer = Pipeline([
    ("outlier", OutlierTransformer(columns=outlier_columns))
])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

drop_transformer = Pipeline([
    ("drop", DropTransformer(columns_to_drop=drop_columns))
])

scale_transformer = Pipeline([
    ("scaler", MinMaxScaler())
])


preprocessor = ColumnTransformer([
    ("temporal", temporal_transformer, temporal_columns),
    ("log", log_transformer, log_columns+temporal_columns),
    ("outlier", outlier_transformer, outlier_columns),
    ("categorical", categorical_transformer, categorical_columns),
    ("drop", drop_transformer, drop_columns),
], remainder="passthrough")  

processing_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("scaler", scale_transformer)
])

In [40]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((283494, 77), (70874, 77), (283494, 1), (70874, 1))

In [41]:
X_train = processing_pipeline.fit_transform(X_train)
X_test = processing_pipeline.transform(X_test)
X_t3 = processing_pipeline.transform(X_t3)

In [42]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_t3.shape

((283494, 85), (283494, 1), (70874, 85), (70874, 1), (88593, 85))

# Dimensionality Reduction


## Low Variance Filter

In [43]:
class VarianceThresholdTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold

    def fit(self, X, y=None):
        variances = np.var(X, axis=0)
        self.keep_indices_ = [i for i, var in enumerate(variances) if var >= self.threshold]

        return self

    def transform(self, X):
        return X[:, self.keep_indices_]

## High Correlation Filter

In [44]:
class HighCorrelationDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.9):
        self.threshold = threshold

    def fit(self, X, y=None):
        corr_matrix = np.corrcoef(X, rowvar=False)
        
        n_features = X.shape[1]
        self.keep_indices_ = []
        for i in range(n_features):
            if all(abs(corr_matrix[i, j]) <= self.threshold for j in self.keep_indices_):
                self.keep_indices_.append(i)

        return self

    def transform(self, X):
        return X[:, self.keep_indices_]

In [45]:
from sklearn.pipeline import Pipeline

dimensionality_reduction_pipeline = Pipeline([
    ('low_variance', VarianceThresholdTransformer(threshold=0.01)), 
    ('high_correlation', HighCorrelationDropper(threshold=0.9)) 
])

X_train_reduced = dimensionality_reduction_pipeline.fit_transform(X_train)
X_test_reduced = dimensionality_reduction_pipeline.transform(X_test)
X_t3_reduced = dimensionality_reduction_pipeline.transform(X_t3)

In [46]:
X_train_reduced.shape, X_test_reduced.shape, X_t3_reduced.shape

((283494, 53), (70874, 53), (88593, 53))

# Optimizacion de parametros

In [47]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_curve, auc
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

def objective(trial, model_name, X_train, y_train, X_test, y_test):
    max_estimators = 350
    max_max_depth = 12
    if model_name == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, max_estimators),
            'max_depth': trial.suggest_int('max_depth', 9, max_max_depth),
            'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.2),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 10),
        }
        model = XGBClassifier(**params, objective='binary:logistic', random_state=100)

        trial.set_user_attr('model_class', model_name)

    elif model_name == 'LightGBM':
        params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, max_estimators),
        'max_depth': trial.suggest_int('max_depth', 3, max_max_depth),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.2),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 10),
        }

        model = LGBMClassifier(**params, random_state=100)

        trial.set_user_attr('model_class', model_name)

    elif model_name == 'RandomForest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, max_estimators),
            'max_depth': trial.suggest_int('max_depth', 3, max_max_depth),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
        }
        model = RandomForestClassifier(**params, random_state=100)

        trial.set_user_attr('model_class', model_name)

    elif model_name == 'KNN':
        params = {
            'n_neighbors': trial.suggest_int('n_neighbors', 3, 15),
            'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        }
        model = KNeighborsClassifier(**params)

        trial.set_user_attr('model_class', model_name)
    else:
        raise ValueError("Invalid model name")

    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    auc_pr = auc(recall, precision)

    return auc_pr


models_dict = {
    'RandomForest': RandomForestClassifier,
    'KNN': KNeighborsClassifier,
    'XGBoost': XGBClassifier,
    'LightGBM': LGBMClassifier
}


models = [ 'XGBoost']
studies = []

for model_name in models:
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name, X_train_reduced, y_train, X_test_reduced, y_test), n_trials=10, n_jobs=8, gc_after_trial=True)
    studies.append(study)

[I 2024-12-12 23:47:57,960] A new study created in memory with name: no-name-1ce448e0-4ea5-4217-bcd2-0557a1836582
[I 2024-12-12 23:48:20,317] Trial 1 finished with value: 0.9078707156845192 and parameters: {'n_estimators': 100, 'max_depth': 12, 'learning_rate': 0.17617884039983228, 'subsample': 0.7229581111541005, 'colsample_bytree': 0.7943973200411776, 'min_child_weight': 5, 'gamma': 5.976123618529431}. Best is trial 1 with value: 0.9078707156845192.
[I 2024-12-12 23:48:26,645] Trial 4 finished with value: 0.8948361235836089 and parameters: {'n_estimators': 124, 'max_depth': 11, 'learning_rate': 0.09859914584790426, 'subsample': 0.5196938647490785, 'colsample_bytree': 0.6204656161307871, 'min_child_weight': 3, 'gamma': 9.229108920429873}. Best is trial 1 with value: 0.9078707156845192.
[I 2024-12-12 23:48:30,542] Trial 2 finished with value: 0.8944570149914479 and parameters: {'n_estimators': 242, 'max_depth': 10, 'learning_rate': 0.08310928271045946, 'subsample': 0.6529833497281272, 

In [48]:
best_models = []
best_model_info = None
best_model_auc = 0
for study in studies:
    study_best_model_info = study.best_trial
    best_models.append(study_best_model_info)
    print(f"Best trial: {study_best_model_info.value}")
    print(f"Best parameters: {study_best_model_info.params}")
    print(f"Model class: {study_best_model_info.user_attrs['model_class']}")
    print()
    if study_best_model_info.value > best_model_auc:
        best_model_auc = study_best_model_info.value
        best_model_info = study_best_model_info

best_model_class_name = best_model_info.user_attrs['model_class']   
best_model_class = models_dict[best_model_class_name]

best_model = best_model_class(**best_model_info.params)
best_model.fit(X_train_reduced, y_train)

y_pred_proba = best_model.predict_proba(X_test_reduced)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
auc_pr = auc(recall, precision)
print(f"Test AUC-PR: {auc_pr}")

y_t3_pred = best_model.predict(X_t3_reduced)
y_t3_pred_proba = best_model.predict_proba(X_t3_reduced)[:, 1]

Best trial: 0.9334174500977
Best parameters: {'n_estimators': 339, 'max_depth': 9, 'learning_rate': 0.16417748131523302, 'subsample': 0.8348572237292549, 'colsample_bytree': 0.8775475995030391, 'min_child_weight': 5, 'gamma': 1.4367786844799535}
Model class: XGBoost

Test AUC-PR: 0.9328124499884725


In [49]:
import mlflow
from mlflow.models import infer_signature

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") 
mlflow.set_experiment(f"Final Model {best_model_class_name}")

with mlflow.start_run():

    mlflow.log_metric("AUC-PR", auc_pr)

    mlflow.xgboost.log_model(best_model, artifact_path="model", registered_model_name=f"Final Model {best_model_class_name}")

c:\Users\fcb11\LabProg\repo\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:49:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/12/12 23:49:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Final Model XGBoost' already exists. Creating a new version of this model...
2024/12/12 23:49:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Final Model XGBoost, version 4


🏃 View run suave-quail-247 at: http://127.0.0.1:5000/#/experiments/965559342375477005/runs/1364ae18eba04b229fa42cc08558ebe9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/965559342375477005


Created version '4' of model 'Final Model XGBoost'.


In [51]:
from zipfile import ZipFile
import os
def generateFiles(predict_data, clf_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    ---------------
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf

    Ouput
    ---------------
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict_proba(predict_data)[:, 1]
    with open('./predictions.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj:
        zipObj.write('predictions.txt')
    os.remove('predictions.txt')

generateFiles(X_t3_reduced, best_model)

In [52]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = best_model.predict(X_test_reduced)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[40216  2325]
 [ 6330 22003]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.90     42541
           1       0.90      0.78      0.84     28333

    accuracy                           0.88     70874
   macro avg       0.88      0.86      0.87     70874
weighted avg       0.88      0.88      0.88     70874

